# Guide for creating machine learning model to predict if credit card shall be approved or denied 

    import libraries that we will use
   

In [10]:
! pip3 install pandas
! pip3 install numpy
! pip3 install sklearn

In [11]:
import numpy as np
import pandas as pd

In [12]:
df1 = pd.read_csv("data.csv")

In [13]:
print (df1)

     YearsEmployed  PriorDefault  Credi1Score  Income  approved
0            1.250             1            1       0         1
1            3.040             1            6     560         1
2            1.500             1            0     824         1
3            3.750             1            5       3         1
4            1.710             1            0       0         1
5            2.500             1            0       0         1
6            6.500             1            0   31285         1
7            0.040             1            0    1349         1
8            3.960             1            0     314         1
9            3.165             1            0    1442         1
10           2.165             0            0       0         1
11           4.335             1            0     200         1
12           1.000             1            0       0         1
13           0.040             0            0    2690         1
14           5.000             1        